 <div style="text-align:center;">
   <span style="color:green; font-size:larger; font-weight:bold;">Vérification de la loi d'Okun dans plusieurs pays du monde</span><br><br>
  <span style="font-weight:bold;">Réalisé par:</span><br>
  <span>NOUBOUSSI GNINTEDEM LUCIE MARIMAR</span><br>
  <span>YOUSRA JEDDOUB</span> <br>
  <span>AMINA MANSEUR</span>
</div>


# <span style="color:green">Introduction</span><br><br> 

Dans cette première partie, l'objectif est de construire les DataFrames nécessaires à notre projet.<br>
La démarche suivie est la suivante :<br>
<div style="margin-left: 20px;">
    <span style="font-weight:bold;">1.</span> Importer les différentes BD en local.<br>
    <span style="font-weight:bold;">2.</span> Construire les Dataframes par l'utilisation des méthodes du module Pandas.<br>
    <span style="font-weight:bold;">3.</span> Les bases de données étant disponibles en ligne ; Automatisation de l'importation en utilisant un url (ceci garantit que les données soient toujours à jour).<br>
    <span style="font-weight:bold;">4.</span> Optimisation du code de facon à ce qu'il soit le plus reproductible possible et ce en définissant des fonctions selon les besoins du projet.
    </div>

# <span style="color:green">I- Importation et installation des packages</span>

Pour plus de clarté et de lisibilité du code, nous déclarons l'ensemble des imports nécessaires dans un fichier distinct "declaration.py". Ce dernier est ainsi appelé au tout début. Si des ajouts, suppressions ou mises à jour des dépendances sont requises, ces dernières sont faites simplement dans le fichier "declaration.py".

In [1]:
from importlib import reload
import monmodule.declarations as d
reload(d)

ModuleNotFoundError: No module named 'monmodule'

# <span style="color:green; ">II- Construction de la base de données</span>

## <span style="color:green; text-align:center;">II-1 Importation de la base GemDataEXTR.Zip via son url de téléchargement</span>

In [ ]:
# L'importation se fait via l'url de téléchargement
url = "https://datacatalogfiles.worldbank.org/ddh-published/0037798/DR0092042/GemDataEXTR.zip?"
d.load(url, "GemDataEXTR.zip")

In [ ]:
# Choix des fichiers à extraire
nom_fichiers = [('Unemployment Rate, seas. adj..xlsx', 'monthly'),
              ('GDP Deflator at Market Prices, LCU.xlsx', 'quarterly')
]

# Extraction des bases dans la mémoire: chomage et PIB déflaté
df_Unemployement, df_GDP = [d.extraire_fichier_zip('GemDataEXTR.zip', nom_fichier, nom_feuille)
                           for nom_fichier, nom_feuille in nom_fichiers]

<div style="margin-left: 20px;">
-L'extraction des données sur le taux de chômage et le taux de croissance du PIB déflaté par pays est désormais complète. <br>
-Dans la prochaine étape, nous procéderons à une exploration rapide de ces données et les fusionnerons pour une analyse plus approfondie.
</div>

### <span style="color:green; text-align:center;">II-1-1 Préparation des bases de données</span>

#### <span style="color:green; text-align:center;">II-1-1-1 Base taux de chômage</span>

In [ ]:
# Visualisation
df_Unemployement.head()

<div style="margin-left: 20px;">
-La 1ère ligne de la base 'df_Unemployement' est vide.<br>
-Ainsi on souhaite commencer à partir de l'année 1994, la ligne correspondant au dernier mois de l'année 1993 sera supprimée.
</div>

In [ ]:
# Suppression des deux prmières lignes
df_Unemployement = df_Unemployement.iloc[2:].copy()

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df_Unemployement.info()

Le type de chaque variable est approprié et correspond aux types attendus. Il s'agit d'un nombre flottant de 64 bits.

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", df_Unemployement.shape[0], "lignes et", df_Unemployement.shape[1], "colonnes.")

##### Detection des doublons

In [ ]:
# Vérification des doublons
print("Nombre total de doublons dans df_Unemployement :",
      df_Unemployement[df_Unemployement.duplicated()].shape[0])

In [ ]:
# Supprimer les doublons (s'ils existent)
df_Unemployement.drop_duplicates(inplace=True)

print("Nombre total de doublons dans df_Unemployement :", 
      df_Unemployement[df_Unemployement.duplicated()].shape[0])

##### Correction des noms des pays

Pour rendre la DF plus lisible, on exprime les noms des pays par leurs codes ISO 3166-1 alpha-3, soit des abréviations à trois (3) lettres.

In [ ]:
# Appliquer la correction sur chaque colonne du DataFrame
df_Unemployement.columns=d.correct_country_name(df_Unemployement.columns)

# Listes des pays détecter
pays=d.detect_country_name(df_Unemployement.columns)

# Base avec colonnes corrigées
df_Unemployement=df_Unemployement[pays]

##### Détection des valeurs manquantes

Pour rendre la DF plus lisible, on exprime les noms des pays par leurs codes ISO 3166-1 alpha-3, soit des abréviations à trois (3) lettres.

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_Unemployement)

Les données consistent en des séries temporelles. Chaque pays disposant de sa propre série temporelle du taux de chômage.<br>
D'après la visualisation ci-dessus, on constate que pour la plupart des pays : 
<div style="margin-left: 20px;">
-Les valeurs manquantes sont en début de la période considérée.<br></div>

Ainsi, nous allons garder uniquement les pays avec au moins 60% d'observations non manquantes.</div>

In [ ]:
# Suppression des pays avec au moins 40% de valeurs manquantes sur la période
df_Unemployement = d.missing(df_Unemployement)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_Unemployement)

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", df_Unemployement.shape[0], "lignes et", df_Unemployement.shape[1], "colonnes.")

À ce stade, on retrouve une DF où uniquement les pays avec suffisamment de données sont representés. 

##### Imputation des valeurs manquantes

L'imputation par la médiane étant plus robuste aux valeurs aberrantes sera privilégiée ici.

In [ ]:
# Imputation des valeurs manquantes 
df_Unemployement = d.fill_missing_with_median(df_Unemployement)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_Unemployement)

Les valeurs manquantes sont totalement imputées. On obtient une DataFrame complète et propre prête à être utilisée pour l'analyse ou la modélisation. 

In [ ]:
# Visualisation
df_Unemployement.head()

In [ ]:
# Colonnes présentes dans Uemploy
pays = df_Unemployement.columns
print('Les pays de la DataFrame  df_Unemployement sont :',pays)

**Précision :** <br>

À ce stade :
<div style="margin-left: 20px;">
-La DataFrame "df_Unemployement" est un tableau à 358 lignes et 31 colonnes. Nous disposons alors d'une série temporelle du taux de chômage de taille 358 observations pour chaque pays parmi les 31 pays.<br>
-L'index de la DF est de type date. Il s'agit en effet de données mensuelles s'étalant sur la période allant de Janvier 1994 à Janvier 2023.</div>

##### Transformation des données mensuelles en données trimestrielles

Plus loin, nous aurrons à réaliser une jointure entre la DataFrame comportant les taux de chomage et celle contenant les taux de croissance du PIB. Les données dont on dispose sur les taux de croissance du PIB sont trimestrielles. Il est alors plus cohérent de transformer les données mensuelles en données trimestrielles pour la DataFrame "df_Unemployement".

In [ ]:
# Grouper par année de 12 mois chacun
df_Unemployement = df_Unemployement\
    .groupby(df_Unemployement.index.year)\
        .filter(lambda x: len(x) == 12)
df_Unemployement = d.pd.DataFrame(df_Unemployement)

L'idée du code précedent est de ne garder que les années pour lequelles les taux de chômage des 12 mois de l'année sont renseignés. L'année 2023, par exemple, sera éliminée car nous disposons que du taux de chomage du 1er mois de cette année.<br>

Le regroupement va se faire par une moyenne arithmétique simple.

In [ ]:
# Transformation
df_Unemployement = df_Unemployement.resample('Q-JAN').mean()

# Ignorer les jours dans l'index
df_Unemployement.index =df_Unemployement.index.to_period('Q').strftime('%Y-Q%q')

# Appliquer le format trimestre au index
df_Unemployement.index = d.pd.to_datetime(df_Unemployement.index+ '-01', format='%Y-Q%m-%d')

# Formater l'index pour obtenir '1994-01' au lieu de '1994-01-01'
df_Unemployement.index = df_Unemployement.index.strftime('%Y-%m')

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", df_Unemployement.shape[0], "lignes et", df_Unemployement.shape[1], "colonnes.")

#### <span style="color:green; text-align:center;">II-1-1-2 Dataframe du taux de croissance du PIB</span>

In [ ]:
# Visualisation
df_GDP.head(5)

La 1ère ligne étant vierge. Cette dernière sera supprimée.

In [ ]:
# Supprimez la ligne avec l'index NaN du DataFrame
df_GDP = df_GDP.drop(df_GDP.index[0])

# L'index de la DF
df_GDP.index

L'index de la DataFrame est de type date, les données sont trimestrielles et s'étalent sur la période allant du premier trimestre de 1994 au dernier trimestre de 2023.

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df_GDP.info()

Le type de données est conforme aux attentes.

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", df_GDP.shape[0], "lignes et", df_GDP.shape[1], "colonnes.")

In [ ]:
# Formater l'index pour obtenir '1994-01' au lieu de '1994-01-01'
df_GDP.index = df_GDP.index.strftime('%Y-%m')

##### Correction des noms des pays

Comme pour la DF "df_unemployment", nous allons remplacer les noms des pays par leurs codes ISO.

In [ ]:
# Appliquer la correction sur chaque colonne du DataFrame
df_GDP.columns = d.correct_country_name(df_GDP.columns)

# Base avec colonnes corrigées presente dans Unemploy
df_GDP=df_GDP[pays]

##### Détection des doublons

In [ ]:
# Vérifier la présence de doublons
print("Nombre total de doublons dans df_GDP :", 
      df_GDP[df_GDP.duplicated()].shape[0])

In [ ]:
# Supprimer les doublons (s'ils existent)
df_GDP.drop_duplicates(inplace=True)

print("Nombre total de doublons dans df_GDP :", 
      df_GDP[df_GDP.duplicated()].shape[0])

##### Détection des valeurs manquantes

Les valeurs manquantes sont visualisées via un diagramme à barre et un heatmap.

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_GDP)

Les données consistent en des séries temporelles. Chaque pays disposant de sa propre série temporelle du taux de croissance du PIB.<br>
D'après la visualisation ci-dessus, on constate que pour la plupart des pays : 
<div style="margin-left: 20px;">
-Les valeurs manquantes sont en début de la période considérée.<br></div>

Ainsi, nous allons garder uniquement les pays avec au moins 60% d'observations non manquantes.</div>

In [ ]:
# Suppression des pays avec au moins 90% des valeurs manquantes sur la période
df_GDP = d.missing(df_GDP)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_GDP)

##### Imputation des valeurs manquantes

On considère l'imputation par la médiane comme méthode d'imputation.

In [ ]:
# Correction des valeurs manquantes 
df_GDP = d.fill_missing_with_median(df_GDP)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_GDP)

Les valeurs manquantes sont totalement imputées. On obtient une DataFrame complète et propre prête à être utilisée pour l'analyse ou la modélisation. 

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", df_GDP.shape[0], "lignes et", df_GDP.shape[1], "colonnes.")

In [ ]:
# Colonnes présentes dans gdp
pays1 = df_GDP.columns
print('Les pays de la DataFrame  df_Unemployement sont :',pays1)

In [ ]:
# Colonnes non présentes dans l'une des DF et pas dans l'autre
print(list(set(pays) - set(pays1))) 

In [ ]:
# Nombre de pays dans chaque DF
print("Le nombre de pays dans 'df_Unemployement' est de", len(pays), ".")
print("Le nombre de pays dans 'df_GDP' est de", len(pays1), ".")

Connaitre le nombre de pays dans chaque DF permet d'avoir une idée sur le nombre de pays qu'on aura au final après la jointure entre 'df_GDP' et 'df_Unemployement. 

**Précision :**

À ce stade :
<div style="margin-left: 20px;">
-La DataFrame "df_GDP" est un tableau à 120 lignes et 30 colonnes. Nous disposons alors d'une série temporelle du taux de croissance du PIB de 120 observations pour chaque pays parmi les 30 pays.<br>
-L'index de la DF est de type date. Il s'agit en effet de données trimestrielles s'étalant sur la période allant du premier trimestre de 1994  jusqu'au dernier trimestre de 2023.</div>

### <span style="color:green; text-align:center;">II-1-2 Fusion des deux bases</span>

In [ ]:
reload(d)
# Tranformation des bases en format long 
dfs = d.transform(df_Unemployement, 'Unemployment_rate')
dfs1 = d.transform(df_GDP, 'GDP_rate')

In [ ]:
# Visualisation
dfs1.head(10)

In [ ]:
# Fusion des bases
df_merge1 = d.pd.merge(dfs, dfs1, on=['YEAR', 'COUNTRY'], how='left') # La clé de jointure est composée des dates et des pays
df_merge1.head()

Suite à la jointure, on obtient une DF contenant en colonne les dates trimestrielles, les pays, les taux de chômage et les taux de croissance du PIB.

## <span style="color:green; text-align:center;">II-2 Importation de la base HNP_Stats_EXCEL.Zip via son url de téléchargement</span>

In [ ]:
# L'importation se fait via l'url de téléchargement.
url = "https://databank.worldbank.org/data/download/HNP_Stats_EXCEL.zip"
d.load(url,"HNP_Stats_EXCEL.zip")

In [ ]:
# Choix des fichiers à extraire
nom_fichiers=[ ('HNP_StatsEXCEL.xlsx', 'Data')]

# Extraction des bases dans la mémoire: chomage et PIB déflaté
Big_data= [d.extraire_fichier_zip('HNP_Stats_EXCEL.zip', nom_fichier, nom_feuille)
                           for nom_fichier, nom_feuille in nom_fichiers]

L'extraction des données sur l'espérance de vie, et le taux de croissance de la population. <br>
Dans la prochaine étape, nous procéderons à une exploration rapide de ces données et les fusionnerons pour une analyse plus approfondie.


### <span style="color:green; text-align:center;">II-2-1 Préparation des bases avant fusion</span>

In [ ]:
Big_data=Big_data[0]
# Visualisation
Big_data.head()

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
Big_data.info()


Le type de chaque variable est approprié et correspond aux types attendus.

#### <span style="color:green; text-align:center;">II-2-1-1 Base de données espérance de vie</span>

In [ ]:
# Extraction des données sur l'esperance de vie 
df_LE = d.extract2(Big_data, 'expectancy', 'total')

In [ ]:
# Visualisation
df_LE.head(2)

In [ ]:
# Construction de la base "life expentancy"
reload(d)
df_LE=d.treat_info(df_LE, pays1)

In [ ]:
# Visualisation
df_LE.head(2)

##### Détection des valeurs manquantes

In [ ]:
# Plot des Nan
d.missing_plot(df_LE)

La DF est complète, aucune valeur manquante n'est constatée.

#### <span style="color:green; text-align:center;">II-2-1-2 Base de données taux de croissance démographique</span>

In [ ]:
# Extracton des données sur la croissance démographique 
reload(d)
df_pop=d.extract2(Big_data, '^Population growth \(annual %\)$','')

In [ ]:
# Visualisation
df_pop.head(2)

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df_pop.info()

In [ ]:
# Construction de la base du taux de croissance démographique
reload(d)
df_pop=d.treat_info(df_pop, pays1)

In [ ]:
# Visualisation
df_pop.tail()

##### Détection des valeurs manquantes

In [ ]:
# Plot
d.missing_plot(df_pop)

La DF est complète, aucune valeur manquante n'est constatée.

### <span style="color:green; text-align:center;">II-2-2 Fusion des deux bases</span>

In [ ]:
reload(d)
# Tranformation des bases en format long 
dfs = d.transform(df_LE, 'life_expentancy')
dfs1 = d.transform(df_pop, 'pop_growth_rate')

In [ ]:
# Visualisation
dfs.head()

In [ ]:
# Fusion des bases
df_merge2 = d.pd.merge(dfs, dfs1, on=['YEAR', 'COUNTRY'], how='left')
df_merge2.head()

In [ ]:
# Fusion merge1 et merge2.
df_merge3 = d.pd.merge(df_merge1, df_merge2, on=['YEAR', 'COUNTRY'], how='left')
df_merge3.head()

In [ ]:
#Exporter le DataFrame au format CSV
df_merge3.to_csv('./monmodule/bases/final_data.csv', index=False)